## Rede Neural Multilayer Perceptron para reconhecimento de padrão:

In [3]:
%matplotlib inline
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter, LogLocator

import numpy as np
import pandas as pd

import random
from math import cos, tanh
from time import sleep

from ipywidgets import *
import ipywidgets as widgets
import re

from IPython.display import display, HTML

########## entradas dos parametros ##########
style = {'description_width': 'initial'}

print('\n\nTaxa de aprendizagem:')
taxa_aprendizagem = widgets.FloatSlider(
    value=0.01,
    min=0.01,
    max=1,
    step=0.01,
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.3f',
)
display(taxa_aprendizagem)

print('\n\nQuantidade de neurônios da camada intermediária:')
qnt_neuronios = widgets.IntSlider(
    value=20,
    min=1,
    max=100,
    step=1,
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)
display(qnt_neuronios)

print('\n\nQuantidade de amostras:')
qnt_amostras = widgets.IntSlider(
    value=50,
    min=1,
    max=1000,
    step=1,
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)
display(qnt_amostras)

print('\n\nCritério de parada:')
parada = widgets.RadioButtons(
    options=['Erro', 'Ciclos'],
    disabled=False, style=style
)
display(parada)

print('\n\nNúmero de ciclos:')
ciclos = widgets.IntSlider(
    value=10,min=1,max=1000,step=1,
    continuous_update=False,
    readout=True, style=style
)
display(ciclos)

print('\n\nErro tolerado:')
erro = widgets.FloatSlider(
    value=0.1,min=0,max=1,step=0.001,
    continuous_update=False,
    readout=True, style=style,
    readout_format='.5f',
)
display(erro)

class RedeMLP():
    def __init__(self, conjunto_entradas, targets, taxa_aprendizagem, parada, ciclos_max, 
                 erro_tolerado, qnt_neuronios, **kwargs):
        self.numero_entradas = 256 #tamanho da imagem
        self.numero_saidas = 10 #0-9
        self.conjunto_entradas = np.array(conjunto_entradas)
        self.targets = np.array(targets)
        self.taxa_aprendizagem = taxa_aprendizagem
        self.taxa_aprendizagem_inicial = taxa_aprendizagem
        self.ciclos = 0
        self.ciclos_max = ciclos_max
        self.parada = parada
        #Quantidade de neuronios seleciondos para camada intermediária
        self.qnt_neuronios_intermediarios = qnt_neuronios
        self.erro_tolerado = erro_tolerado
        #Pesos das ligações entre as camadas inicial e intermediária; intermediária e saída
        self.pesos_ini_inter, self.pesos_inter_saida = self.__inicializa_pesos()
        #Iniciliza os bias da camada intermediária
        self.bias_intermediario = np.asmatrix(np.array([random.uniform(-0.5,0.5) for saida in range(qnt_neuronios)]))
        #Inicializa o bias da camada de saida
        self.bias_saida = np.asmatrix(np.array([random.uniform(-0.5,0.5) for i in range(self.numero_saidas)]))
        self.erro = 0
        
    def __inicializa_pesos(self):
        pesos_ini_inter = []
        pesos_inter_saida = []
        
        #mxn = onde m é o número de entradas e n é o número de neurônios da camada intermediária
        for i in range(self.numero_entradas):
            peso_aux = []
            for j in range(self.qnt_neuronios_intermediarios):
                peso_aux.append(random.uniform(-0.5,0.5))
            pesos_ini_inter.append(peso_aux)
        
        #nxk onde n é o número de neurônios da camada intermediária e k é o número de saídas
        for i in range(self.qnt_neuronios_intermediarios):
            peso_aux = []
            for i in range(self.numero_saidas):
                peso_aux.append(random.uniform(-0.5,0.5))
            pesos_inter_saida.append(peso_aux)
            
        #print("Tam peso ini_inter", len(pesos_ini_inter[0]))
        #print("Tam peso inter_saida", len(pesos_inter_saida[0]))
        return np.array(pesos_ini_inter), np.array(pesos_inter_saida)
    
    def __f_ativacao(self, matriz):
        #print("ZIN", zin)
        out = []
        dim = matriz.shape #dimensão da matriz
        for i in range(dim[0]):
            aux = []
            for j in range(dim[1]):
                aux.append(tanh(matriz.item(i,j)))
            out.append(aux)
                
        return np.matrix(np.array(out))
    
    def __calcula_saida__(self, saida):
        dists_euclidiana = []
        for i in range(10):
            calculo = np.sum(np.power(np.power((self.targets[i]-saida),2),0.5))
            dists_euclidiana.append(calculo)
        menor_valor = np.amin(dists_euclidiana)
        id_menor_valor = dists_euclidiana.index(menor_valor)
        return np.asmatrix(self.targets[id_menor_valor]);
    
    def __normaliza_saida__(self, saida):
        saida[saida >= 0.5] = 1
        saida[saida < 0.5] = 0
        return saida
        
    
    def treinamento(self):
        print("#Parametros da função")
        treinada = False
        erros = []
        
        while not treinada:
            i = 0 
            self.erro = 0
            
            for entrada_i in self.conjunto_entradas:
                ####################Etapa de Forward#################################
                ##############Camada inicial para intermediária
                #resultado = 1xn, onde n é o número de neurônios da camada intermediária
                
                zin = (np.asmatrix(entrada_i) @ self.pesos_ini_inter) + self.bias_intermediario
                z = self.__f_ativacao(zin)

                ##############Camada intermediária para saída
                yin = (z @ self.pesos_inter_saida) + self.bias_saida
                y = self.__f_ativacao(yin)
                #y = self.__calcula_saida__(y)
                #print("y", y)
                
                
                #####################Etapa de backward
                #Cálculo do erro
                self.erro += 0.5 * np.sum(np.power((self.targets[i] - y), 2))
                
                #########cálculo dos parâmetros para correção dos erros da camada intermediária para camada de saída
                #Calculo dos parâmetros para correção dos pesos e do bias
                #np.multiply para se fazer uma multiplicação item por item
                deltak = np.multiply((self.targets[i] - y).T,(1 + y).T)
                deltak = np.multiply(deltak, (1 - y).T)
                
                
                i+=1#ERRADO
                
                #Parâmetro para correção dos pesos
                delta_pesos_inter_saida = deltak @ z
                delta_pesos_inter_saida = self.taxa_aprendizagem * delta_pesos_inter_saida
                
                #Parâmetro para correção do bias
                delta_bias_saida = np.multiply(self.taxa_aprendizagem, deltak)
                
                #print('delta pesos de saida: ', delta_pesos_inter_saida)
                #print('delta bias de saida: ', delta_bias_saida)
                
                #########cálculo dos parâmetros para correção dos erros da camada entrada para camada de intermediária
                #Calculo dos parâmetros para correção dos pesos e do bias
                delta_in = deltak.T @ self.pesos_inter_saida.T
                
                
                deltaj = np.multiply(delta_in, (1+z))
                deltaj = np.multiply(deltaj, (1-z))
                #print('delta_in: ', delta_in)
                #print('deltaj: ', deltaj)
                
                #Parâmetro para correção dos pesos
                delta_pesos_ini_inter = deltaj.T @ np.asmatrix(entrada_i)
                delta_pesos_ini_inter = np.multiply(self.taxa_aprendizagem, delta_pesos_ini_inter)
                
                delta_bias_intermediario = np.multiply(self.taxa_aprendizagem, deltaj)
                #print('delta pesos de intermediário: ', delta_pesos_ini_inter)
                #print('delta bias de intermediário: ', delta_bias_intermediario)

                #Correção dos pesos da camada intermediária para camada de saída e bias da camada de saída
                self.pesos_inter_saida = self.pesos_inter_saida + delta_pesos_inter_saida.T
                
                self.bias_saida = (self.bias_saida + delta_bias_saida.T)
                
                #print("Pesos corrigidos da camada saída:", self.pesos_inter_saida)
                #print("bias corrigidos da camada saída:", self.bias_saida)
                
                #Correção dos pesos da camada intermediária para camada de saída e bias da camada de intermediária
                
                self.pesos_ini_inter = self.pesos_ini_inter + delta_pesos_ini_inter.T
                self.bias_intermediario = (self.bias_intermediario + delta_bias_intermediario)
                #print("Pesos corrigidos da camada intermediária:", self.pesos_ini_inter)
                #print("bias corrigidos da camada intermediária:", self.bias_intermediario)
                
                #Ajuste da taxa de aprendizagem
            self.erro = (1/900)*self.erro
            erros.append(self.erro)
            print('erro', self.erro)
            if (self.parada == 'ciclos' and self.ciclos >= self.ciclos_max) \
                or (self.parada == 'erro' and self.erro <= self.erro_tolerado):
                #print('erro final: ', self.erro)
                treinada = True
            else:
                self.ciclos += 1
                
        return list(range(0, self.ciclos+1)), erros
    
    def operacao(self, conjunto_testes):
        acertos = 0
        i = 0
        for entrada_i in self.conjunto_entradas:
            ####################Etapa de Forward#################################
            ##############Camada inicial para intermediária
            #resultado = 1xn, onde n é o número de neurônios da camada intermediária
                
            zin = (np.asmatrix(entrada_i) @ self.pesos_ini_inter) + self.bias_intermediario
            z = self.__f_ativacao(zin)

            ##############Camada intermediária para saída
            yin = (z @ self.pesos_inter_saida) + self.bias_saida
            y = self.__f_ativacao(yin)
            print('yin0', y)
            y = self.__normaliza_saida__(y)
            #y = self.__calcula_saida__(y)
            print('y', np.asmatrix(y))
            print('t', np.asmatrix(self.targets[i]))
            if(np.array_equal(np.asmatrix(y), np.asmatrix(self.targets[i]))):
                acertos += 1
            i+=1

        return acertos

##### definindo as entradas a partir do arquivo digitostreinamento900.txt #####
def get_inputs_from_txt():
    inputs = []
    file = open('digitos/digitostreinamento900.txt', 'r')
    for line in file :
        line = line.split(" ")
        ipt = []
        for element in line:
            if element != "":
                ipt.append(float(element))
        inputs.append(ipt)
    file.close()
    return np.array(inputs)
##############################################

def get_obvs(k, m):
    semente = np.array([[1]])
    obvs = []
    for i in range(k):
        aux_obv = []
        for j in range(2**i):
            comb1 = np.array(np.concatenate((semente[j], semente[j])))
            comb2 = np.array(np.concatenate((semente[j], -1*semente[j])))
            aux_obv.append(np.array(comb1))
            aux_obv.append(np.array(comb2))
        semente = aux_obv
        obvs = aux_obv
        
    targets = []
    for i in range(90):
        for j in range(10):
            targets.append(obvs[j])
        
    return np.array(targets)

def get_targets():
    targets_aux = pd.read_csv('targets.csv').values
    targets = []
    for i in range(90):
        for j in range(10):
            targets.append(list(targets_aux[j]))
        
    return np.array(targets)

def processar():
    conjunto_entradas = get_inputs_from_txt()
    #targets = get_obvs(4,1)
    targets = get_targets()
    #print("Conjunto de entradas:", conjunto_entradas[0])
    mlp = RedeMLP(
        conjunto_entradas = conjunto_entradas, 
        targets = targets, 
        taxa_aprendizagem = taxa_aprendizagem.value,
        parada = parada.value.lower(),
        ciclos_max = ciclos.value,
        erro_tolerado = erro.value,
        qnt_neuronios = qnt_neuronios.value
    )

    #print("Entradas: ", mlp.conjunto_entradas)
    lista_ciclos, lista_erros = mlp.treinamento() 
    acertos = mlp.operacao(conjunto_entradas)
    print(acertos)
    

widgets.interact_manual.opts['manual_name'] = 'Treinar a rede' # muda texto do botao
interact_manual(processar); # metodo a executar quando pressionar o botao



Taxa de aprendizagem:


FloatSlider(value=0.01, continuous_update=False, max=1.0, min=0.01, readout_format='.3f', step=0.01)



Quantidade de neurônios da camada intermediária:


IntSlider(value=20, continuous_update=False, min=1)



Quantidade de amostras:


...



Número de ciclos:


IntSlider(value=10, continuous_update=False, max=1000, min=1, style=SliderStyle(description_width='initial'))



Erro tolerado:


FloatSlider(value=0.1, continuous_update=False, max=1.0, readout_format='.5f', step=0.001, style=SliderStyle(d…

interactive(children=(Button(description='Treinar a rede', style=ButtonStyle()), Output()), _dom_classes=('wid…

In [34]:
teste = np.array([ [1,2,3], [3,4,5], [6,8,7]])
teste1 = np.array([1,1,1])
print(teste-teste1)

ValueError: ndarray is not contiguous

In [42]:
np.linspace(1, 10, 10)

array([ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10.])

In [70]:
v0j = read_matrix_in_file('392-87/v0j.txt')
vij = read_matrix_in_file('392-87/vij.txt')
w0j = read_matrix_in_file('392-87/w0j.txt')
wij = read_matrix_in_file('392-87/wij.txt')
print(v0j.shape)
print(vij.shape)
print(w0j.shape)
print(wij.shape)

[[-0.22354054 -0.61118557  0.03716968 -0.2317244  -0.4144495  -0.05752555
  -0.86821146  0.3014956   0.04521217  0.19666127 -0.15475972 -0.25142094
   0.07633184  0.4075155  -0.06897117  0.1118349   1.01426525 -0.25805678
   0.24749779  0.14361168 -0.14910899 -0.60085866  0.02685383  0.3085577
  -0.42033196  0.03933269  0.23263577  0.2446363  -0.45036318 -0.31725727
  -0.33673836  0.47271518  0.4897469  -0.43765284 -0.16178674]]
(256, 35)
(1, 35)
(35, 16)


In [69]:
def read_matrix_in_file(path):
    file = open(path)
    matrix = []
    for line in file:
        matrix.append(list(map(float, line.split(','))))
    return np.asmatrix(matrix)
    